In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import glob
import matplotlib.pyplot as plt
import seaborn as sns

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [9]:
def load_compile_epc(path_to_cleaned):

    list_paths = glob.glob(path_to_cleaned) #path_to_cleaned = "./../Data/Energy_Performance_Certificate/cerificates_processed/*.csv"

    data = pd.DataFrame()
    for i in range(len(glob.glob(path_to_cleaned))):
        x = pd.read_csv(list_paths[i])
        data = pd.concat([data,x])

    #data = gpd.GeoDataFrame(data, geometry = gpd.points_from_xy(data.LONGITUDE,data.LATITUDE), crs = "EPSG:4326")

    return(data)

def winsor(variable):
    upper_percentile = np.percentile(variable, 99)
    lower_percentile = np.percentile(variable, 1)
    new_var = np.where(variable > upper_percentile, upper_percentile, variable)
    new_var = np.where(new_var < lower_percentile, lower_percentile, new_var)
    return(new_var)

def essen_clean(data):
    # eliminate rows before 01/05/2013
    data['LODGEMENT_DATE'] = pd.to_datetime(data['LODGEMENT_DATE'],format = '%Y-%m-%d')
    data = data[data['LODGEMENT_DATE'] >= '2013-05-01 00:00:00']
    # eliminate not geolocated rows
    data = data[~data['DataZone'].isna()]

    return(data)



In [5]:
lookup = pd.read_csv('./../../Volumes/Extreme_SSD/WORK/MAV/UPRN/uprn_datazones_lookup.csv')

# ENERGY

In [6]:
ener = load_compile_epc('./../../Volumes/Extreme_SSD/WORK/EPC/scottish_EPC/cleaned_energy/*')
ener.head()

,UPRN,LODGEMENT_DATE,ENERGY_CONSUMPTION_CURRENT,CURRENT_ENERGY_EFFICIENCY,CURRENT_ENERGY_RATING,ENERGY_CONSUMPTION_POTENTIAL,MAINS_GAS_FLAG,LATITUDE,LONGITUDE
0,35000326.0,2014-09-18,469.0,67,D,164.0,Y,56.124597,-3.813194
1,35000379.0,2014-07-04,175.0,71,C,82.0,Y,56.108770,-3.790594
2,35000527.0,2014-07-03,231.0,67,D,186.0,Y,56.121653,-3.793252
3,35000632.0,2014-07-18,210.0,68,D,137.0,Y,56.123294,-3.798460
4,35000947.0,2014-09-23,294.0,57,D,105.0,Y,56.127769,-3.818717


In [7]:
ener = ener.merge(lookup, how = 'left', on = 'UPRN')
ener.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1435091 entries, 0 to 1435090
Data columns (total 10 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   UPRN                          1435091 non-null  float64
 1   LODGEMENT_DATE                1435091 non-null  object 
 2   ENERGY_CONSUMPTION_CURRENT    1435091 non-null  float64
 3   CURRENT_ENERGY_EFFICIENCY     1435091 non-null  int64  
 4   CURRENT_ENERGY_RATING         1435090 non-null  object 
 5   ENERGY_CONSUMPTION_POTENTIAL  1435091 non-null  float64
 6   MAINS_GAS_FLAG                60049 non-null    object 
 7   LATITUDE                      1434901 non-null  float64
 8   LONGITUDE                     1434901 non-null  float64
 9   DataZone                      1434881 non-null  object 
dtypes: float64(5), int64(1), object(4)
memory usage: 109.5+ MB


In [10]:
ener = essen_clean(ener)

In [12]:
# adjust the energy related variables
ener = ener.loc[:,['CURRENT_ENERGY_RATING', 'CURRENT_ENERGY_EFFICIENCY','ENERGY_CONSUMPTION_CURRENT', 'ENERGY_CONSUMPTION_POTENTIAL', 'MAINS_GAS_FLAG','DataZone']]
ener['count'] = 1
dict_ener = {'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'G':7,'INVALID!': np.nan}
ener['CURRENT_ENERGY_RATING_numbered'] = ener['CURRENT_ENERGY_RATING'].map(dict_ener)

In [13]:
ener.head()

,CURRENT_ENERGY_RATING,CURRENT_ENERGY_EFFICIENCY,ENERGY_CONSUMPTION_CURRENT,ENERGY_CONSUMPTION_POTENTIAL,MAINS_GAS_FLAG,DataZone,count,CURRENT_ENERGY_RATING_numbered
0,D,67,469.0,164.0,Y,S01007475,1,4.0
1,C,71,175.0,82.0,Y,S01007465,1,3.0
2,D,67,231.0,186.0,Y,S01007467,1,4.0
3,D,68,210.0,137.0,Y,S01007470,1,4.0
4,D,57,294.0,105.0,Y,S01007474,1,4.0


In [14]:
ener_g = ener[ener['MAINS_GAS_FLAG'] == 'Y']

In [15]:
ener['ENERGY_CONSUMPTION_CURRENT'] = winsor(ener['ENERGY_CONSUMPTION_CURRENT'])
ener_g['ENERGY_CONSUMPTION_CURRENT'] = winsor(ener_g['ENERGY_CONSUMPTION_CURRENT'])

/var/folders/t5/57qdsp617xj8dd1by4td7fl40000gq/T/ipykernel_51387/3467100616.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ener_g['ENERGY_CONSUMPTION_CURRENT'] = winsor(ener_g['ENERGY_CONSUMPTION_CURRENT'])


In [16]:
# aggregate full data
agg = round(ener.groupby(['DataZone']
                  ).aggregate({'CURRENT_ENERGY_EFFICIENCY':['median','mean'],
                              'CURRENT_ENERGY_RATING_numbered':['median','mean'],
                              'ENERGY_CONSUMPTION_CURRENT':['median','mean'],
                              'ENERGY_CONSUMPTION_POTENTIAL':['median','mean'],
                              'count':['count']}).reset_index(),2)
agg.columns = [c[0] + "_" + c[1] for c in agg.columns]
agg = agg.rename(columns = {'DataZone_':'DataZone'})
# aggregate the gas only data
agg_g = round(ener_g.groupby(['DataZone']
                  ).aggregate({'CURRENT_ENERGY_EFFICIENCY':['median','mean'],
                              'CURRENT_ENERGY_RATING_numbered':['median','mean'],
                              'ENERGY_CONSUMPTION_CURRENT':['median','mean'],
                              'ENERGY_CONSUMPTION_POTENTIAL':['median','mean'],
                              'count':['count']}).reset_index(),2)
agg_g.columns = [c[0] + "_" + c[1] for c in agg_g.columns]
agg_g = agg_g.rename(columns = {'DataZone_':'DataZone'})

In [ ]:
agg.to_csv('./../../Volumes/Extreme_SSD/WORK/EPC/scottish_EPC/DZ_energy.csv')
agg_g.to_csv('./../../Volumes/Extreme_SSD/WORK/EPC/scottish_EPC/DZ_GAS_energy.csv')

# HEAT

In [23]:
heat = load_compile_epc('./../../Volumes/Extreme_SSD/WORK/EPC/scottish_EPC/cleaned_heat/*')
heat.head()

,UPRN,LODGEMENT_DATE,MAINHEAT_DESCRIPTION,SECONDHEAT_DESCRIPTION,MAINS_GAS_FLAG,NUMBER_OPEN_FIREPLACES,LATITUDE,LONGITUDE
0,35000326.0,2014-09-18,Electric storage heaters,"Room heaters, electric",Y,0.0,56.124597,-3.813194
1,35000379.0,2014-07-04,"Boiler and radiators, mains gas","Room heaters, electric",Y,0.0,56.108770,-3.790594
2,35000527.0,2014-07-03,"Boiler and radiators, mains gas","Room heaters, mains gas",Y,0.0,56.121653,-3.793252
3,35000632.0,2014-07-18,"Boiler and radiators, mains gas",None,Y,0.0,56.123294,-3.798460
4,35000947.0,2014-09-23,"Boiler and radiators, mains gas",None,Y,0.0,56.127769,-3.818717


In [25]:
heat = heat.merge(lookup, how = 'left', on = 'UPRN')
heat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1435091 entries, 0 to 1435090
Data columns (total 9 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   UPRN                    1435091 non-null  float64
 1   LODGEMENT_DATE          1435091 non-null  object 
 2   MAINHEAT_DESCRIPTION    1435091 non-null  object 
 3   SECONDHEAT_DESCRIPTION  1348136 non-null  object 
 4   MAINS_GAS_FLAG          60049 non-null    object 
 5   NUMBER_OPEN_FIREPLACES  1434505 non-null  float64
 6   LATITUDE                1434901 non-null  float64
 7   LONGITUDE               1434901 non-null  float64
 8   DataZone                1434881 non-null  object 
dtypes: float64(4), object(5)
memory usage: 98.5+ MB


In [26]:
heat = essen_clean(heat)
heat['count'] = 1

def return_binary_heat(source_var, str_name):
    var = source_var.str.find(str_name)
    var = np.where(var > 0, 1,0)
    return var

#heat['electric'] = return_binary_heat(source_var = heat['MAINHEAT_DESCRIPTION'], 
                                          str_name =  'electric')


## mainheating

In [29]:
keywords = ['electric', 'trydan']
# Create a regex pattern for the keywords
pattern = '|'.join(keywords)
# Use str.contains to create the binary column
heat['MAIN_electric'] = heat['MAINHEAT_DESCRIPTION'].str.contains(pattern, case=False, na=True).astype(int)
keywords = ['wood', 'coal', 'LPG', 
' oil', 'anthracite', 'smokeless',
'bio', 'B30K', 'solid fuel', 'coal', 'wood',
 'olew', 'choed', 'LNG']
# Create a regex pattern for the keywords
pattern = '|'.join(keywords)
# Use str.contains to create the binary column
heat['MAIN_solid_fuel'] = heat['MAINHEAT_DESCRIPTION'].str.contains(pattern, case=False, na=True).astype(int)

In [30]:
heat['MAIN_other'] = np.where(((heat['MAIN_electric'] == 0) | (heat['MAIN_solid_fuel'] == 0)), 1,0)
heat['MAIN_electric_other'] = np.where((heat['MAIN_electric'] & heat['MAIN_other']), 1, 0)
heat['MAIN_electric_solid'] = np.where((heat['MAIN_electric'] & heat['MAIN_solid_fuel']), 1, 0)
heat['MAIN_solid_other'] = np.where((heat['MAIN_solid_fuel'] & heat['MAIN_other']), 1, 0)

In [31]:
for i in [ 'MAIN_electric','MAIN_solid_fuel',
       'MAIN_other', 'MAIN_electric_other', 'MAIN_electric_solid',
       'MAIN_solid_other']: 
       heat[i] = np.where(heat['MAINHEAT_DESCRIPTION'].isna(), np.nan, heat[i] )

## secondheating

In [33]:
keywords = ['electric', 'trydan']
# Create a regex pattern for the keywords
pattern = '|'.join(keywords)
# Use str.contains to create the binary column
heat['SECOND_electric'] = heat['SECONDHEAT_DESCRIPTION'].str.contains(pattern, case=False, na=True).astype(int)
keywords = ['wood', 'coal', 'LPG', 
' oil', 'anthracite', 'smokeless',
'bio', 'B30K', 'solid fuel', 'coal', 'wood',
 'olew', 'choed', 'LNG']
# Create a regex pattern for the keywords
pattern = '|'.join(keywords)
# Use str.contains to create the binary column
heat['SECOND_solid_fuel'] = heat['SECONDHEAT_DESCRIPTION'].str.contains(pattern, case=False, na=True).astype(int)

In [34]:
heat['SECOND_other'] = np.where(((heat['SECOND_electric'] == 0) | (heat['SECOND_solid_fuel'] == 0)), 1,0)
heat['SECOND_electric_other'] = np.where((heat['SECOND_electric'] & heat['SECOND_other']), 1, 0)
heat['SECOND_electric_solid'] = np.where((heat['SECOND_electric'] & heat['SECOND_solid_fuel']), 1, 0)
heat['SECOND_solid_other'] = np.where((heat['SECOND_solid_fuel'] & heat['SECOND_other']), 1, 0)

In [35]:
for i in [ 'SECOND_electric','SECOND_solid_fuel',
       'SECOND_other', 'SECOND_electric_other', 'SECOND_electric_solid',
       'SECOND_solid_other']: 
       heat[i] = np.where(heat['SECONDHEAT_DESCRIPTION'].isna(), np.nan, heat[i] )

## other the heat variables

In [36]:
heat['NUMBER_OPEN_FIREPLACES_binary'] = np.where(heat['NUMBER_OPEN_FIREPLACES'] >=1, 1, 0)

In [37]:
heat_g = heat[heat['MAINS_GAS_FLAG'] == 'Y']

## aggregate

In [38]:
agg = round(heat.groupby(['DataZone']
                  ).aggregate({'NUMBER_OPEN_FIREPLACES':['sum','median','mean'],
                              'NUMBER_OPEN_FIREPLACES_binary':['sum'],
                              'MAIN_electric':['sum'],
                                'MAIN_solid_fuel':['sum'],
                                'MAIN_other':['sum'], 
                                'MAIN_electric_other':['sum'], 
                                'MAIN_electric_solid':['sum'],
                                'MAIN_solid_other':['sum'], 
                                'SECOND_electric':['sum'], 
                                'SECOND_solid_fuel':['sum'],
                                'SECOND_other':['sum'], 
                                'SECOND_electric_other':['sum'], 
                                'SECOND_electric_solid':['sum'],
                                'SECOND_solid_other':['sum'],
                              'count':['count']}).reset_index(),2)
agg.columns = [c[0] + "_" + c[1] for c in agg.columns]
agg = agg.rename(columns = {'DataZone_':'DataZone'})

agg_g = round(heat_g.groupby(['DataZone']
                  ).aggregate({'NUMBER_OPEN_FIREPLACES':['sum','median','mean'],
                              'NUMBER_OPEN_FIREPLACES_binary':['sum'],
                              'MAIN_electric':['sum'],
                                'MAIN_solid_fuel':['sum'],
                                'MAIN_other':['sum'], 
                                'MAIN_electric_other':['sum'], 
                                'MAIN_electric_solid':['sum'],
                                'MAIN_solid_other':['sum'], 
                                'SECOND_electric':['sum'], 
                                'SECOND_solid_fuel':['sum'],
                                'SECOND_other':['sum'], 
                                'SECOND_electric_other':['sum'], 
                                'SECOND_electric_solid':['sum'],
                                'SECOND_solid_other':['sum'],
                              'count':['count']}).reset_index(),2)
agg_g.columns = [c[0] + "_" + c[1] for c in agg_g.columns]
agg_g = agg_g.rename(columns = {'DataZone_':'DataZone'})

In [45]:
agg.to_csv('./../../Volumes/Extreme_SSD/WORK/EPC/scottish_EPC/DZ_heat.csv')
agg_g.to_csv('./../../Volumes/Extreme_SSD/WORK/EPC/scottish_EPC/DZ_GAS_heat.csv')

In [42]:
del heat, heat_g, agg, agg_g

# HOUSE

In [185]:
house = load_compile_epc('./../../Volumes/Extreme_SSD/WORK/EPC/scottish_EPC/cleaned_house/*')
house.head()

,UPRN,LODGEMENT_DATE,TOTAL_FLOOR_AREA,CONSTRUCTION_AGE_BAND,MAINS_GAS_FLAG,TENURE,BUILT_FORM,PROPERTY_TYPE,LATITUDE,LONGITUDE
0,35000326.0,2014-09-18,43,1984-1991,Y,owner-occupied,End-Terrace,Flat,56.124597,-3.813194
1,35000379.0,2014-07-04,78,1976-1983,Y,rented (social),Mid-Terrace,House,56.108770,-3.790594
2,35000527.0,2014-07-03,60,1930-1949,Y,owner-occupied,Semi-Detached,Flat,56.121653,-3.793252
3,35000632.0,2014-07-18,71,1919-1929,Y,rented (social),Semi-Detached,Flat,56.123294,-3.798460
4,35000947.0,2014-09-23,81,1965-1975,Y,owner-occupied,Detached,House,56.127769,-3.818717


In [186]:
house = house.merge(lookup, how = 'left', on = 'UPRN')
house.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1435091 entries, 0 to 1435090
Data columns (total 11 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   UPRN                   1435091 non-null  float64
 1   LODGEMENT_DATE         1435091 non-null  object 
 2   TOTAL_FLOOR_AREA       1435091 non-null  int64  
 3   CONSTRUCTION_AGE_BAND  1299300 non-null  object 
 4   MAINS_GAS_FLAG         60049 non-null    object 
 5   TENURE                 1435091 non-null  object 
 6   BUILT_FORM             1430077 non-null  object 
 7   PROPERTY_TYPE          1435091 non-null  object 
 8   LATITUDE               1434901 non-null  float64
 9   LONGITUDE              1434901 non-null  float64
 10  DataZone               1434881 non-null  object 
dtypes: float64(3), int64(1), object(7)
memory usage: 120.4+ MB


In [187]:
house = essen_clean(house)
house['count'] = 1
house = house.reset_index()

In [188]:
house['BUILT_FORM'] = np.where(house['BUILT_FORM'] == 'NO DATA!', np.nan,house['BUILT_FORM'] )

In [189]:
dict_ten = {'owner-occupied':'Owner-occupied', 'rented (private)':'Rented (private)', 'rented (social)':'Rented (social)', 'unknown':np.nan}
house['TENURE'] = house['TENURE'].map(dict_ten)

In [194]:
dict_age = {'1984-1991':'1976-2002', 
            '1976-1983':'1976-2002', 
            '1930-1949':'1930-1949', 
            '1919-1929':'before_1929', 
            '1965-1975':'1950-1975',
            '1992-1998':'1976-2002', 
            '1950-1964':'1950-1975', 
            '2003-2007':'2003_onwards', 
            'before 1919':'before_1929', 
            '1999-2002':'1976-2002',
            '2008 onwards':'2003_onwards'}
house['CONSTRUCTION_AGE_BAND'] = house['CONSTRUCTION_AGE_BAND'].map(dict_age)

In [195]:
gas = {'N':0,'Y':1}
house['MAINS_GAS_FLAG'] = house['MAINS_GAS_FLAG'].map(gas)

In [196]:
house_g = house[house['MAINS_GAS_FLAG'] == 1]

In [197]:
PROP_TYPE = pd.pivot(house.loc[:,['PROPERTY_TYPE','UPRN','DataZone','count','index']], 
         index = ['index','DataZone'],columns='PROPERTY_TYPE', values='count'
         ).reset_index().loc[:,['DataZone','Bungalow','Flat','House','Maisonette','Park home']
                                ].groupby('DataZone').sum().reset_index(

                                ).rename(columns = {'Bungalow':'Bungalow_sum','Flat':'Flat_sum',
                                                                                            'House':'House_sum','Maisonette':'Maisonette_sum',
                                                                                            'Park home':'Park_home_sum'})
PROP_TYPE_g = pd.pivot(house_g.loc[:,['PROPERTY_TYPE','UPRN','DataZone','count','index']], 
         index = ['index','DataZone'],columns='PROPERTY_TYPE', values='count'
         ).reset_index().loc[:,['DataZone','Bungalow','Flat','House','Maisonette']
                                ].groupby('DataZone').sum().reset_index(

                                ).rename(columns = {'Bungalow':'Bungalow_sum','Flat':'Flat_sum',
                                                                                            'House':'House_sum','Maisonette':'Maisonette_sum'})

In [198]:
BUILD = pd.pivot(house.loc[:,['BUILT_FORM','UPRN','DataZone','count','index']], 
         index = ['index','DataZone'],columns='BUILT_FORM', values='count'
         ).reset_index().loc[:,['DataZone','Detached', 'End-Terrace', 'Mid-Terrace', 'Semi-Detached']
                                ].groupby('DataZone').sum().reset_index(

                                ).rename(columns = {'Detached':'Detached_sum', 'End-Terrace':'End-Terrace_sum',
                                                     'Mid-Terrace':'Mid-Terrace_sum', 'Semi-Detached':'Semi-Detached_sum'})
BUILD_g = pd.pivot(house_g.loc[:,['BUILT_FORM','UPRN','DataZone','count','index']], 
         index = ['index','DataZone'],columns='BUILT_FORM', values='count'
         ).reset_index().loc[:,['DataZone','Detached', 'End-Terrace', 'Mid-Terrace', 'Semi-Detached']
                                ].groupby('DataZone').sum().reset_index(

                                ).rename(columns = {'Detached':'Detached_sum', 'End-Terrace':'End-Terrace_sum',
                                                     'Mid-Terrace':'Mid-Terrace_sum', 'Semi-Detached':'Semi-Detached_sum'})

In [199]:
house['CONSTRUCTION_AGE_BAND'].unique()

array(['1976-2002', '1930-1949', 'before_1929', '1950-1975',
       '2003_onwards', nan], dtype=object)

In [200]:
AGE = pd.pivot(house.loc[:,['CONSTRUCTION_AGE_BAND','UPRN','DataZone','count','index']], 
         index = ['index','DataZone'],columns='CONSTRUCTION_AGE_BAND', values='count'
         ).reset_index().loc[:,['DataZone','1976-2002', '1930-1949', 'before_1929', '1950-1975',
       '2003_onwards']
                                ].groupby('DataZone').sum().reset_index(

                                ).rename(columns = {'1976-2002':'1976-2002_sum', '1930-1949':'1930-1949_sum', 
                                                    'before_1929':'before_1929_sum', '1950-1975':'1950-1975_sum','2003_onwards':'2003_onwards_sum'})

AGE_g = pd.pivot(house_g.loc[:,['CONSTRUCTION_AGE_BAND','UPRN','DataZone','count','index']], 
         index = ['index','DataZone'],columns='CONSTRUCTION_AGE_BAND', values='count'
         ).reset_index().loc[:,['DataZone','1976-2002', '1930-1949', 'before_1929', '1950-1975','2003_onwards']
                                ].groupby('DataZone').sum().reset_index(

                                ).rename(columns = {'1976-2002':'1976-2002_sum', '1930-1949':'1930-1949_sum', 
                                                    'before_1929':'before_1929_sum', '1950-1975':'1950-1975_sum','2003_onwards':'2003_onwards_sum'})

In [201]:
TENURE = pd.pivot(house.loc[:,['TENURE','UPRN','DataZone','count','index']], 
         index = ['index','DataZone'],columns='TENURE', values='count'
         ).reset_index().loc[:,['DataZone','Owner-occupied', 'Rented (private)', 'Rented (social)']
                                ].groupby('DataZone').sum().reset_index(

                                ).rename(columns = {'Owner-occupied':'Owner-occupied_sum', 'Rented (private)':'Rented (private)_sum', 'Rented (social)':'Rented (social)_sum'})
TENURE_g = pd.pivot(house_g.loc[:,['TENURE','UPRN','DataZone','count','index']], 
         index = ['index','DataZone'],columns='TENURE', values='count'
         ).reset_index().loc[:,['DataZone','Owner-occupied', 'Rented (private)', 'Rented (social)']
                                ].groupby('DataZone').sum().reset_index(

                                ).rename(columns = {'Owner-occupied':'Owner-occupied_sum', 'Rented (private)':'Rented (private)_sum', 'Rented (social)':'Rented (social)_sum'})

In [202]:
agg = round(house.groupby(['DataZone']
                  ).aggregate({'TOTAL_FLOOR_AREA':['median','mean'],
                               'MAINS_GAS_FLAG':['sum'],
                              'count':['count']}).reset_index(),2)
agg.columns = [c[0] + "_" + c[1] for c in agg.columns]
agg = agg.rename(columns = {'DataZone_':'DataZone'})

agg = agg.merge(TENURE, on = 'DataZone'
          ).merge(BUILD, on = 'DataZone'
                  ).merge(PROP_TYPE, on = 'DataZone'
                          ).merge(AGE, on = 'DataZone')

agg_g = round(house_g.groupby(['DataZone']
                  ).aggregate({'TOTAL_FLOOR_AREA':['median','mean'],
                               'MAINS_GAS_FLAG':['sum'],
                              'count':['count']}).reset_index(),2)
agg_g.columns = [c[0] + "_" + c[1] for c in agg_g.columns]
agg_g = agg_g.rename(columns = {'DataZone_':'DataZone'})

agg_g = agg_g.merge(TENURE_g, on = 'DataZone'
          ).merge(BUILD_g, on = 'DataZone'
                  ).merge(PROP_TYPE_g, on = 'DataZone'
                          ).merge(AGE_g, on = 'DataZone')

In [203]:
agg.to_csv('./../../Volumes/Extreme_SSD/WORK/EPC/scottish_EPC/DZ_house.csv')
agg_g.to_csv('./../../Volumes/Extreme_SSD/WORK/EPC/scottish_EPC/DZ_GAS_house.csv')

In [73]:
del house, house_g, agg, agg_g

# LOSS

In [204]:
loss = load_compile_epc('./../../Volumes/Extreme_SSD/WORK/EPC/scottish_EPC/cleaned_loss/*')
loss.head()

,UPRN,LODGEMENT_DATE,ROOF_DESCRIPTION,EXTENSION_COUNT,MAINS_GAS_FLAG,MULTI_GLAZE_PROPORTION,PHOTO_SUPPLY,LATITUDE,LONGITUDE
0,35000326.0,2014-09-18,"Pitched, 100 mm loft insulation",0,Y,0,Array: Roof Area: 0%; |,56.124597,-3.813194
1,35000379.0,2014-07-04,"Pitched, 250 mm loft insulation",0,Y,100,Array: Roof Area: 0%; |,56.108770,-3.790594
2,35000527.0,2014-07-03,(another dwelling above),0,Y,100,Array: Roof Area: 0%; |,56.121653,-3.793252
3,35000632.0,2014-07-18,(another dwelling above),0,Y,100,Array: Roof Area: 0%; |,56.123294,-3.798460
4,35000947.0,2014-09-23,"Pitched, 250 mm loft insulation",0,Y,100,Array: Roof Area: 0%; |,56.127769,-3.818717


In [205]:
loss = loss.merge(lookup, how = 'left', on = 'UPRN')
loss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1435091 entries, 0 to 1435090
Data columns (total 10 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   UPRN                    1435091 non-null  float64
 1   LODGEMENT_DATE          1435091 non-null  object 
 2   ROOF_DESCRIPTION        1435091 non-null  object 
 3   EXTENSION_COUNT         1435091 non-null  int64  
 4   MAINS_GAS_FLAG          60049 non-null    object 
 5   MULTI_GLAZE_PROPORTION  1435091 non-null  int64  
 6   PHOTO_SUPPLY            1388213 non-null  object 
 7   LATITUDE                1434901 non-null  float64
 8   LONGITUDE               1434901 non-null  float64
 9   DataZone                1434881 non-null  object 
dtypes: float64(3), int64(2), object(5)
memory usage: 109.5+ MB


In [206]:
loss = essen_clean(loss)
loss['count'] = 1

In [207]:
keywords = ['insulat', 'inswleid']
# Create a regex pattern for the keywords
pattern = '|'.join(keywords)
# Use str.contains to create the binary column
loss['Roof_insulati'] = loss['ROOF_DESCRIPTION'].str.contains(pattern, case=False, na=True).astype(int)

keywords = ['thatch']
# Create a regex pattern for the keywords
pattern = '|'.join(keywords)
# Use str.contains to create the binary column
loss['Roof_thatch'] = loss['ROOF_DESCRIPTION'].str.contains(pattern, case=False, na=True).astype(int)

In [208]:
li_0 = ['Array: Roof Area: 0%; Connection: not applicable (FGHRS or no PV);  | ',
       'Array: Roof Area: 0%;  | ',
       'Array: Roof Area: 0%; Connection: not recorded;  | ',
       'Array: Roof Area: %;  | ']

loss['PHOTO_SUPPLY_bi'] = np.where(loss['PHOTO_SUPPLY'].isin(li_0), 0 , 1)

In [209]:
loss_g = loss[loss['MAINS_GAS_FLAG'] == 'Y']

In [211]:
agg = round(loss.groupby(['DataZone']
                  ).aggregate({'MULTI_GLAZE_PROPORTION':['median','mean'],
                               'EXTENSION_COUNT':['median','mean'],
                               'PHOTO_SUPPLY_bi': ['sum'],
                               'Roof_insulati':['sum'],
                               'Roof_thatch':['sum'],
                              'count':['count']}).reset_index(),2)
agg.columns = [c[0] + "_" + c[1] for c in agg.columns]
agg = agg.rename(columns = {'DataZone_':'DataZone'})

agg_g = round(loss_g.groupby(['DataZone']
                  ).aggregate({'MULTI_GLAZE_PROPORTION':['median','mean'],
                               'EXTENSION_COUNT':['median','mean'],
                               'PHOTO_SUPPLY_bi': ['sum'],
                               'Roof_insulati':['sum'],
                               'Roof_thatch':['sum'],
                              'count':['count']}).reset_index(),2)
agg_g.columns = [c[0] + "_" + c[1] for c in agg_g.columns]
agg_g = agg_g.rename(columns = {'DataZone_':'DataZone'})

In [212]:
agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6974 entries, 0 to 6973
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   DataZone                       6974 non-null   object 
 1   MULTI_GLAZE_PROPORTION_median  6974 non-null   float64
 2   MULTI_GLAZE_PROPORTION_mean    6974 non-null   float64
 3   EXTENSION_COUNT_median         6974 non-null   float64
 4   EXTENSION_COUNT_mean           6974 non-null   float64
 5   PHOTO_SUPPLY_bi_sum            6974 non-null   int64  
 6   Roof_insulati_sum              6974 non-null   int64  
 7   Roof_thatch_sum                6974 non-null   int64  
 8   count_count                    6974 non-null   int64  
dtypes: float64(4), int64(4), object(1)
memory usage: 490.5+ KB


In [213]:
agg_g.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6204 entries, 0 to 6203
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   DataZone                       6204 non-null   object 
 1   MULTI_GLAZE_PROPORTION_median  6204 non-null   float64
 2   MULTI_GLAZE_PROPORTION_mean    6204 non-null   float64
 3   EXTENSION_COUNT_median         6204 non-null   float64
 4   EXTENSION_COUNT_mean           6204 non-null   float64
 5   PHOTO_SUPPLY_bi_sum            6204 non-null   int64  
 6   Roof_insulati_sum              6204 non-null   int64  
 7   Roof_thatch_sum                6204 non-null   int64  
 8   count_count                    6204 non-null   int64  
dtypes: float64(4), int64(4), object(1)
memory usage: 436.3+ KB


In [214]:
agg.to_csv('./../../Volumes/Extreme_SSD/WORK/EPC/scottish_EPC/DZ_loss.csv')
agg_g.to_csv('./../../Volumes/Extreme_SSD/WORK/EPC/scottish_EPC/DZ_GAS_loss.csv')

In [81]:
del loss, loss_g, agg, agg_g

# OTHER

In [215]:
other = load_compile_epc('./../../Volumes/Extreme_SSD/WORK/EPC/scottish_EPC/cleaned_other/*')
other.head()

,UPRN,LODGEMENT_DATE,FLOOR_LEVEL,MAINS_GAS_FLAG,TRANSACTION_TYPE,PROPERTY_TYPE,LATITUDE,LONGITUDE
0,35000326.0,2014-09-18,top floor,Y,marketed sale,Flat,56.124597,-3.813194
1,35000379.0,2014-07-04,NaN,Y,none of the above,House,56.108770,-3.790594
2,35000527.0,2014-07-03,ground floor,Y,assessment for green deal,Flat,56.121653,-3.793252
3,35000632.0,2014-07-18,ground floor,Y,rental,Flat,56.123294,-3.798460
4,35000947.0,2014-09-23,NaN,Y,none of the above,House,56.127769,-3.818717


In [216]:
other = other.merge(lookup, how = 'left', on = 'UPRN')
other = essen_clean(other)
other['count'] = 1

In [217]:
other['TOP_FLOOR'] = np.where(other['FLOOR_LEVEL']=='top floor',1,0)

In [218]:
other['TOP_FLOOR_FLAT'] = np.where((other['TOP_FLOOR'] == 1) & (other['PROPERTY_TYPE'].isin(['Flat','Maisonette'])), 1,0)

In [219]:
dict_tr = {'marketed sale' :'other',
            'ECO assessment' :'G_deal', 
            'rental' :'other',
              'rental (private)' :'other',
       'new dwelling' :'other', 
       'FiT application' :'G_deal', 
       'none of the above' :'other',
       'rental (social)' :'other', 
       'assessment for green deal' :'G_deal',
       'Stock condition survey' :'other', 
       'non marketed sale' :'other',
       'not sale or rental' :'other',
         'RHI application' :'G_deal', 
         'following green deal' :'G_deal',
       'Stock Condition Survey' :'other', 
       'unknown' :'other',
       'not recorded':'other',
       'rental (private) - this is for backwards compatibility only and should not be used' :'other',
       'not recorded - this is for backwards compatibility only and should not be used' :'other',
       'rental (social) - this is for backwards compatibility only and should not be used':'other'}

other['TRAN_TYPE'] =  other['TRANSACTION_TYPE'].map(dict_tr)
other = other.reset_index()

In [220]:
other_g = other[other['MAINS_GAS_FLAG'] == 'Y']

In [221]:
TRAN_TYPE = pd.pivot(other.loc[:,['TRAN_TYPE','UPRN','DataZone','count','index']], 
         index = ['index','DataZone'],columns='TRAN_TYPE', values='count'
         ).reset_index().loc[:,['DataZone','other', 'G_deal']
                                ].groupby('DataZone').sum().reset_index(

                                ).rename(columns = {'other':'TRANS_TYPE_other_sum','G_deal':'TRANS_TYPE_govdeal_sum'})

TRAN_TYPE_g = pd.pivot(other_g.loc[:,['TRAN_TYPE','UPRN','DataZone','count','index']], 
         index = ['index','DataZone'],columns='TRAN_TYPE', values='count'
         ).reset_index().loc[:,['DataZone','other', 'G_deal']
                                ].groupby('DataZone').sum().reset_index(

                                ).rename(columns = {'other':'TRANS_TYPE_other_sum','G_deal':'TRANS_TYPE_govdeal_sum'})

In [222]:
agg = round(other.groupby(['DataZone']
                  ).aggregate({'TOP_FLOOR_FLAT':'sum',
                              'count':['count']}).reset_index(),2)
agg.columns = [c[0] + "_" + c[1] for c in agg.columns]
agg = agg.rename(columns = {'DataZone_':'DataZone'})

agg_g = round(other_g.groupby(['DataZone']
                  ).aggregate({'TOP_FLOOR_FLAT':'sum',
                              'count':['count']}).reset_index(),2)
agg_g.columns = [c[0] + "_" + c[1] for c in agg_g.columns]
agg_g = agg_g.rename(columns = {'DataZone_':'DataZone'})

In [223]:
agg = agg.merge(TRAN_TYPE, on = 'DataZone')
agg_g = agg_g.merge(TRAN_TYPE_g, on = 'DataZone')

In [224]:
agg_g.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6204 entries, 0 to 6203
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   DataZone                6204 non-null   object 
 1   TOP_FLOOR_FLAT_sum      6204 non-null   int64  
 2   count_count             6204 non-null   int64  
 3   TRANS_TYPE_other_sum    6204 non-null   float64
 4   TRANS_TYPE_govdeal_sum  6204 non-null   float64
dtypes: float64(2), int64(2), object(1)
memory usage: 242.5+ KB


In [225]:
agg.to_csv('./../../Volumes/Extreme_SSD/WORK/EPC/scottish_EPC/DZ_other.csv')
agg_g.to_csv('./../../Volumes/Extreme_SSD/WORK/EPC/scottish_EPC/DZ_GAS_other.csv')